# Model Building

In [17]:
# Import libraries
import pandas as pd
import boto3
import sagemaker
from pyathena import connect
from sagemaker.session import Session
from sagemaker.feature_store.feature_group import FeatureGroup

In [2]:
# View created feature groups
!aws sagemaker list-feature-groups

{
    "FeatureGroupSummaries": [
        {
            "FeatureGroupName": "foodlens-products-feature-group-30-14-49-50",
            "FeatureGroupArn": "arn:aws:sagemaker:us-east-1:654654380268:feature-group/foodlens-products-feature-group-30-14-49-50",
            "CreationTime": "2025-09-30T14:49:51.125000+00:00",
            "FeatureGroupStatus": "Created",
            "OfflineStoreStatus": {
                "Status": "Active"
            }
        }
    ]
}


In [46]:
# Change this to your feature group (ex. "foodlens-products-feature-group-30-14-49-50")
feature_group_name = None
assert feature_group_name is not None

In [18]:
# Initialize SageMaker session and Feature Group
# Setup boto and sagemaker session
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
feature_group = FeatureGroup(name=feature_group_name, sagemaker_session=sess)

# Initialize athena query staging
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

# Create connection to Athena database
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [40]:
# Get the name of the unique identifier column
record_id_name = feature_group.describe()['RecordIdentifierFeatureName']

# Get the Athena table name
query = feature_group.athena_query()
table_name = query.table_name

print(f"Athena Table Name: {table_name}")
print(f"Record Identifier: {record_id_name}")

Athena Table Name: foodlens_products_feature_group_30_14_49_50_1759243791
Record Identifier: code


In [42]:
# Split data into training/testing/validation/production sets

# Base query template
# We use xxhash64 for its speed. It creates a deterministic hash for each record ID.
query_template = f"""
SELECT *
FROM "{table_name}"
WHERE MOD(from_big_endian_64(xxhash64(CAST("{record_id_name}" AS VARBINARY))), 100)
"""

# --- Training Data (40%) ---
train_query_string = query_template + " BETWEEN 0 AND 39"
train_query = feature_group.athena_query()
train_query.run(query_string=train_query_string, output_location=f'{s3_staging_dir}/feature_store_queries/train/')
train_query.wait()
train_df = train_query.as_dataframe()

# --- Production Data (40%) ---
prod_query_string = query_template + " BETWEEN 40 AND 79"
prod_query = feature_group.athena_query()
prod_query.run(query_string=prod_query_string, output_location=f'{s3_staging_dir}/feature_store_queries/prod/')
prod_query.wait()
prod_df = prod_query.as_dataframe()

# --- Test Data (10%) ---
test_query_string = query_template + " BETWEEN 80 AND 89"
test_query = feature_group.athena_query()
test_query.run(query_string=test_query_string, output_location=f'{s3_staging_dir}/feature_store_queries/test/')
test_query.wait()
test_df = test_query.as_dataframe()

# --- Validation Data (10%) ---
validation_query_string = query_template + " BETWEEN 90 AND 99"
validation_query = feature_group.athena_query()
validation_query.run(query_string=validation_query_string, output_location=f'{s3_staging_dir}/feature_store_queries/validation/')
validation_query.wait()
validation_df = validation_query.as_dataframe()

# Verify the shapes of the resulting DataFrames
print(f"Training set shape:   {train_df.shape}")
print(f"Production set shape: {prod_df.shape}")
print(f"Test set shape:       {test_df.shape}")
print(f"Validation set shape: {validation_df.shape}")

Training set shape:   (20586, 30)
Production set shape: (19795, 30)
Test set shape:       (5017, 30)
Validation set shape: (5067, 30)


In [43]:
# View training data samples
train_df.head(3)

,code,product_name,nova_group,additives_n,ingredients_n,nutriscore_score,energy_100g,salt_100g,carbohydrates_100g,cholesterol_100g,...,nova_group_100g,nutrition_score_fr_100g,vitamin_a_100g,iron_100g,calcium_100g,vitamin_c_100g,eventtime,write_time,api_invocation_time,is_deleted
0,0769745001238,Bars,4,1,15,12,1703.0,0.1275,55.93,0.0,...,4.0,12.0,0.001017,0.0061,0.339,0.0041,1.759244e+09,2025-09-30 14:57:54.124,2025-09-30 14:52:56.000,False
1,0642526682027,Country house,3,0,10,9,2092.0,0.2925,40.00,0.0,...,3.0,9.0,0.000000,0.0048,0.200,0.0000,1.759244e+09,2025-09-30 14:57:54.124,2025-09-30 14:52:56.000,False
2,0811627020414,Gourmet select,4,2,28,33,2092.0,0.5825,70.00,0.0,...,4.0,33.0,0.000000,0.0000,0.067,0.0000,1.759244e+09,2025-09-30 14:57:54.124,2025-09-30 14:52:57.000,False


In [ ]:
# Perform Standardization scaling for regression (ONLY ON TRAINING DATA)
